Załadowanie pełnej ścieżki do projektu

In [1]:
import sys

if sys.platform == "win32":
    # w tym miejscu należy wstawić ścieżkę do projektu na lokalnym dysku
    sys.path.append("D:/DevSpace/Projects/Research/SeizureDetection")
elif sys.platform == "linux":
    sys.path.append("/mnt/d/gniazdko/SeizureDetection")

Import potrzebnych bibliotek i modułów

In [2]:
import os
import torch
from torchsummary import summary

from core.utils import ProjectManager, ConfigManager
from core.utils import load_model_input_data

from core.machine_learning import (
    split_data_by_proportions,
    create_dataloaders,
    evaluate_model_performance,
    print_classification_report,
    train_and_validate_model
)

from core.machine_learning.model_builder import R2Plus1DConvNet
from core.machine_learning.accuracy import BinaryAccuracy

from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss

d:\DevSpace\Projects\Research\research_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Inicjalizacja ProjectManagera i ConfigManagera do obsługi ścieżek i koniguracji

In [3]:
project_manager = ProjectManager()

configs_directory_path = project_manager.get_configs_directory_path()
primary_data_path = project_manager.get_primary_data_path()
model_directory_path = project_manager.get_model_data_path()

config_manager = ConfigManager(configs_directory_path)

parameters = config_manager.load_config("parameters_machine_learning")
data_parameters = parameters["data_parameters"]
model_parameters = parameters["model_parameters"]

Załadowanie danych

In [4]:
input_data = load_model_input_data(primary_data_path, parameters)

In [5]:
data_splits = split_data_by_proportions(input_data, data_parameters)

# tworzenie dataloderów jest mocno zależne od projektu i problemu
# polcam zapoznać się z funkcją create_dataloders i klasą CustomDataset
train_data, valid_data, test_data = create_dataloaders(data_splits, data_parameters)

Tworzenie instancji modelu

In [6]:
model = R2Plus1DConvNet(
    in_channels=1,
    num_classes=2,
    dropout=0.25
)

model = model.to("cuda")

In [7]:
summary(model, input_data=(1, 18, 128, 128));

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 18, 64, 64]      --
|    └─Conv2Plus1D: 2-1                  [-1, 16, 18, 64, 64]      --
|    |    └─Conv2d: 3-1                  [-1, 24, 64, 64]          1,200
|    |    └─Conv1d: 3-2                  [-1, 16, 18]              1,168
|    └─BatchNorm3d: 2-2                  [-1, 16, 18, 64, 64]      32
|    └─ReLU: 2-3                         [-1, 16, 18, 64, 64]      --
├─ResidualBlock: 1-2                     [-1, 16, 9, 32, 32]       --
|    └─ConvolutionalBlock: 2-4           [-1, 16, 9, 32, 32]       --
|    |    └─Sequential: 3-3              [-1, 16, 9, 32, 32]       13,992
|    |    └─Conv2Plus1D: 3-4             [-1, 16, 9, 32, 32]       280
|    |    └─ReLU: 3-5                    [-1, 16, 9, 32, 32]       --
|    └─IdentityBlock: 2-5                [-1, 16, 9, 32, 32]       --
|    |    └─Sequential: 3-6              [-1, 16, 9, 32, 32]       13,992


Inicjalizowanie niezbędnych obiektów

In [8]:
loss_fn = CrossEntropyLoss()
accuracy_fn = BinaryAccuracy()

optimizer = SGD(
    params=model.parameters(),
    lr=model_parameters["learning_rate"],
    weight_decay=model_parameters["weight_decay"])
    
lr_scheduler = StepLR(
    optimizer,
    step_size=model_parameters["step_size"],
    gamma=model_parameters["gamma"])

In [9]:
# polecam zapoznać się z train_and_validate_model oraz z funkcją perform_step -> plik engine.py
results = train_and_validate_model(
    model=model,
    train_dataloader=train_data,
    valid_dataloader=valid_data,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    num_epochs=10)

Training has started...



 10%|█         | 1/10 [00:09<01:29,  9.96s/it]


Epoch: 1
Training loss: 0.7899 | Validation loss: 0.7436
Training accuracy: 0.4983 | Validation accuracy: 0.4703



 20%|██        | 2/10 [00:20<01:20, 10.02s/it]


Epoch: 2
Training loss: 0.7735 | Validation loss: 0.7434
Training accuracy: 0.4992 | Validation accuracy: 0.4703



...


Epoch: 8
Training loss: 0.7424 | Validation loss: 0.7365
Training accuracy: 0.4986 | Validation accuracy: 0.4703



 90%|█████████ | 9/10 [01:27<00:09,  9.73s/it]


Epoch: 9
Training loss: 0.7393 | Validation loss: 0.7347
Training accuracy: 0.4987 | Validation accuracy: 0.4703



100%|██████████| 10/10 [01:38<00:00,  9.81s/it]


Epoch: 10
Training loss: 0.7374 | Validation loss: 0.7334
Training accuracy: 0.4988 | Validation accuracy: 0.4703

Training of 'seizure detection model' completed after 10 epochs.



Ocena modelu

In [10]:
report = evaluate_model_performance(model, test_data)
print_classification_report(report)

-- -- Model Evaluation: -- --
Loss: 0.7090
Accuracy: 0.5228

Confusion Matrix:
 [[  0 293]
 [  0 321]]

Precision Score: 0.5228
Recall Score: 1.0000
F1 Score: 0.6866



In [11]:
# zapisanie wag modelu
# torch.save(model.state_dict(), ścieżka_do_katalogu)

Załadowanie wag modelu

In [12]:
model.load_state_dict(torch.load(
    os.path.join(
        os.path.join(
            model_directory_path,
            "classifier_2025-03-14_19-03"
        ),
        "model.pth"
    )
))

<All keys matched successfully>

In [13]:
report = evaluate_model_performance(model, test_data)
print_classification_report(report)

-- -- Model Evaluation: -- --
Loss: 0.2672
Accuracy: 0.9072

Confusion Matrix:
 [[272  21]
 [ 36 285]]

Precision Score: 0.9314
Recall Score: 0.8879
F1 Score: 0.9091

